## Importing libraries

In [5]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
import sys
import matplotlib.pyplot as plt
sys.path.append("../")

In [6]:
X_train = pd.read_csv('dataset/X_train.csv')
y_train = pd.read_csv('dataset/y_train.csv')

In [7]:
X_train.head()

,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_count,credits,keywords
0,Alev Alev,Drama-Thriller,tr,NaN,0.664,Erler Film,01-01-1984,0,0,118.0,Released,NaN,2,Tarık Akan-Gülşen Bubikoğlu-Cüneyt Arkın-Çiğde...,pregnancy-model-sea captain-businessman-illega...
1,Those Who Work,Drama,fr,Frank a man of action who worked his way up al...,4.174,Box Productions-Novak Prod-Office Fédéral de l...,04-10-2018,0,0,102.0,Released,NaN,67,Olivier Gourmet-Adèle Bochatay-Delphine Bibet-...,NaN
2,Driven,NaN,en,In a world of adrenaline and speed a quadriple...,0.600,NaN,13-12-2019,0,0,17.0,Released,NaN,0,NaN,NaN
3,Netherlands Documentary,NaN,en,Sex Drugs & Other Taboo Topics The World Is To...,0.600,NaN,27-02-2020,0,0,60.0,Released,NaN,0,NaN,NaN
4,Utta Danella - Der Verlobte meiner besten Freu...,Drama,de,Katharina and Elena are best friends. After a ...,1.152,NaN,03-04-2009,0,0,88.0,Released,NaN,2,Henriette Richter-Röhl-Ina Paule Klink-Robert ...,NaN


In [8]:
X_train.isnull().sum()

title                        4
genres                  223398
original_language            0
overview                122950
popularity                   0
production_companies    402472
release_date             60287
budget                       0
revenue                      0
runtime                  40194
status                       0
tagline                 641108
vote_count                   0
credits                 231217
keywords                534209
dtype: int64

## First lets work on the Numeric part

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752678 entries, 0 to 752677
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   title                 752674 non-null  object 
 1   genres                529280 non-null  object 
 2   original_language     752678 non-null  object 
 3   overview              629728 non-null  object 
 4   popularity            752678 non-null  float64
 5   production_companies  350206 non-null  object 
 6   release_date          692391 non-null  object 
 7   budget                752678 non-null  int64  
 8   revenue               752678 non-null  int64  
 9   runtime               712484 non-null  float64
 10  status                752678 non-null  object 
 11  tagline               111570 non-null  object 
 12  vote_count            752678 non-null  int64  
 13  credits               521461 non-null  object 
 14  keywords              218469 non-null  object 
dtype

In [10]:
le = preprocessing.LabelEncoder()
le.fit(list(X_train['original_language']))
org_lang = le.transform(list(X_train['original_language']))
le.fit(X_train['status'])
status = le.transform(list(X_train['status']))
X_train.drop(columns=['original_language','status'] , inplace= True)

In [11]:
X_train.insert(1, 'status', status)
X_train.insert(0, 'original_language', org_lang)
X_train.head()

,original_language,title,status,genres,overview,popularity,production_companies,release_date,budget,revenue,runtime,tagline,vote_count,credits,keywords
0,147,Alev Alev,4,Drama-Thriller,NaN,0.664,Erler Film,01-01-1984,0,0,118.0,NaN,2,Tarık Akan-Gülşen Bubikoğlu-Cüneyt Arkın-Çiğde...,pregnancy-model-sea captain-businessman-illega...
1,41,Those Who Work,4,Drama,Frank a man of action who worked his way up al...,4.174,Box Productions-Novak Prod-Office Fédéral de l...,04-10-2018,0,0,102.0,NaN,67,Olivier Gourmet-Adèle Bochatay-Delphine Bibet-...,NaN
2,31,Driven,4,NaN,In a world of adrenaline and speed a quadriple...,0.600,NaN,13-12-2019,0,0,17.0,NaN,0,NaN,NaN
3,31,Netherlands Documentary,4,NaN,Sex Drugs & Other Taboo Topics The World Is To...,0.600,NaN,27-02-2020,0,0,60.0,NaN,0,NaN,NaN
4,27,Utta Danella - Der Verlobte meiner besten Freu...,4,Drama,Katharina and Elena are best friends. After a ...,1.152,NaN,03-04-2009,0,0,88.0,NaN,2,Henriette Richter-Röhl-Ina Paule Klink-Robert ...,NaN


### Building the neural network for numeric part of the code

In [12]:
number_columns = ['vote_average', 'vote_count', 'runtime', 'revenue', 'budget', 'popularity']
X_train.isnull().sum()

original_language            0
title                        4
status                       0
genres                  223398
overview                122950
popularity                   0
production_companies    402472
release_date             60287
budget                       0
revenue                      0
runtime                  40194
tagline                 641108
vote_count                   0
credits                 231217
keywords                534209
dtype: int64

In [13]:
X_train['runtime'] = X_train['runtime'].fillna(X_train['runtime'].mean())

In [14]:
X_train.isna().sum()

original_language            0
title                        4
status                       0
genres                  223398
overview                122950
popularity                   0
production_companies    402472
release_date             60287
budget                       0
revenue                      0
runtime                      0
tagline                 641108
vote_count                   0
credits                 231217
keywords                534209
dtype: int64

In [15]:
X_train = X_train[['vote_count', 'runtime', 'revenue', 'budget', 'popularity', 'original_language', 'status']]

In [16]:
X_train.head()

,vote_count,runtime,revenue,budget,popularity,original_language,status
0,2,118.0,0,0,0.664,147,4
1,67,102.0,0,0,4.174,41,4
2,0,17.0,0,0,0.600,31,4
3,0,60.0,0,0,0.600,31,4
4,2,88.0,0,0,1.152,27,4


In [17]:
X_train.isnull().sum()

vote_count           0
runtime              0
revenue              0
budget               0
popularity           0
original_language    0
status               0
dtype: int64

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [19]:
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 34,177
Trainable params: 34,177
Non-trainable params: 0
_________________________________________________________________


In [20]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hX_train5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [21]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [ ]:
# with tf.device('/GPU:0'):
NN_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.1, callbacks=callbacks_list)